# Are university towns' housing prices less affected by recession ?


The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

## 1. Get the list of university towns

Import data from Wikipedia page on college towns. Convert the abbriviation of State names to their full name.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

In [15]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [16]:
# Returns a DataFrame of towns and the states they are in from the university_towns.txt list
def get_list_of_university_towns():
   
    with open('university_towns.txt') as f:
        lst=[]
        State=''
        RegionName=''
        for line in f:          
            line=line.strip()
            if '[edit]' in line:
                State=line[:-6]
            else:
                RegionName=line.split('(')[0].strip()
                lst.append([State,RegionName])
        
    df_uni=pd.DataFrame(lst,columns=['State','RegionName'])
    
    return df_uni
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


## 2. Find the recession bottom after 2000

Import data from Bureau of Economic Analysis, pinpoint the recession period, then get the year of recession bottom.

* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.

In [17]:
# import data of GDP 
def r_data():
    rs=pd.read_excel('gdplev.xls', skiprows=5)
    rs=rs[rs['Unnamed: 4'].notnull()]
    rs=rs[rs['Unnamed: 4'].str[:4].astype('int', errors='ignore')>=2000].iloc[:,np.r_[4,6]]
    rs=rs.rename(columns={"Unnamed: 4":"Quarter","GDP in billions of chained 2009 dollars.1":"GDP_chained_2009"})
    rs['diff']=rs.GDP_chained_2009.diff().fillna(0)
    
    return rs
    
# Returns the year and quarter of the recession start time
def get_recession_start():
        
    rs=r_data()
    r_start=[]
    r_end=[]
    recession=False
    
    for i in range(1, len(rs)-1):
        if (rs.iloc[i-1,2]<0) and (rs.iloc[i,2]<0) and not recession:
            r_start.append(rs.iloc[i-1,0])
            recession=True
        elif (rs.iloc[i-1,2]>0) and (rs.iloc[i,2]>0) and recession:
            #r_end.append(rs.iloc[i-1,0])
            recession=False
    
    return r_start[0]

get_recession_start()

'2008q3'

In [18]:
# Returns the year and quarter of the recession end time
def get_recession_end():
    
    rs=r_data()
    r_start=[]
    r_end=[]
    recession=False
    
    for i in range(1, len(rs)-1):
        if (rs.iloc[i-1,2]<0) and (rs.iloc[i,2]<0) and not recession:
            #r_start.append(rs.iloc[i-1,0])
            recession=True
        elif (rs.iloc[i-1,2]>0) and (rs.iloc[i,2]>0) and recession:
            r_end.append(rs.iloc[i,0])
            recession=False
    
    return r_end[0]    
       

get_recession_end()

'2009q4'

In [19]:
# Returns the year and quarter of the recession bottom time
def get_recession_bottom():
 
    rcsn=r_data().set_index('Quarter')[get_recession_start():get_recession_end()]    
    btn=rcsn.index[rcsn.GDP_chained_2009==rcsn.GDP_chained_2009.min()][0]    
    return btn

get_recession_bottom()

'2009q2'

## 3. Prepare housing data

Extract data after 2000. Then group data by quarter.

In [20]:
# Convert qarter format to "yyyyq#"
def fun_quarter(text):
    if '-' in text:       
        year,month=text.split('-')
        
        if float(month)<4:
            month='q1'
        elif float(month)<7:
            month='q2'
        elif float(month)<10:
            month='q3'
        else:
            month='q4'
        text=year+month
            
    return text


# Converts the housing data to quarters and returns it as mean values in a dataframe
def convert_housing_data_to_quarters():

    
    hp=pd.read_csv('City_Zhvi_AllHomes.csv')
    star_num=hp.columns.get_loc('2000-01')
    hp=hp.iloc[:,np.r_[1,2,star_num:len(hp.columns)]]
    star_num=hp.columns.get_loc('2000-01')
    hp['State']=hp['State'].replace(states)
    a=hp.set_index(['State','RegionName']).sort_index()
    hp.columns=hp.columns.to_series().apply(fun_quarter)    
    hp_1=hp.iloc[:,2:].groupby(level=0,axis=1).mean()
    hp_2=hp.iloc[:,0:2]
    hp=pd.merge(hp_2,hp_1,left_index=True,right_index=True)
    hp=hp.set_index(['State','RegionName']).sort_index()
    return hp


convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002q2  ...         2014q2         2014q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

## 4. Peform T-test

In this analysis, `price_ratio`, the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom, is used to measure the influence of recession on housing price.

$$price\_ratio=\frac{housing\>price\>at\>the\>quarter\>before\>recession}{housing\>price\>at\>the\>recession\>bottom}$$

$H_{0}$: The price ratio of houses in university towns is equal to the price ratio in other areas

$H_{1}$: The price ratio of houses in university towns is unequal to the price ratio of houses in other areas.


In [31]:
# Find the quarter before the recession started
def before_recession():
    b4_rs=get_recession_start().split('q')
    b4_rs=[int(i) for i in b4_rs]
    if b4_rs[1]-1<=0:
        b4_rs[0]=b4_rs[0]-1
        b4_rs[1]=4
        b4_rs=str(b4_rs[0])+'q'+str(b4_rs[1])
    else:
        b4_rs[1]=b4_rs[1]-1
        b4_rs=str(b4_rs[0])+'q'+str(b4_rs[1])
    return b4_rs
        
        
    
# Create a new variable "price ratio" and perform t-test at significance level of 1%

def run_ttest():
    
    
    house_price_all=convert_housing_data_to_quarters().loc[:,[before_recession(),get_recession_bottom()]]
    
    house_price_all['ratio']=house_price_all['2009q2']/house_price_all['2008q2']
    
    lst_uni=get_list_of_university_towns().set_index(['State','RegionName'])
    lst_uni['uni_house']=True
    
    hp_uni=pd.merge(house_price_all,lst_uni,left_index=True,right_index=True, how='inner')
    hp_notuni=house_price_all.drop(hp_uni.index)
    p=ttest_ind(hp_uni['ratio'],hp_notuni['ratio'],nan_policy='omit')[1]
    
    if p<0.01:
        difference='reject'
        if hp_uni['ratio'].mean()>hp_notuni['ratio'].mean():
            better='University town'
        else:
            better='Non-university town'
        note=". {}'s housing prices are less affected by recession. ".format(better)
    else:
                 
        difference='cannot reject'
        note='.'
    
    return 'We {0} the null hypothesis{1}'.format(difference,note)

run_ttest()

"We reject the null hypothesis. University town's housing prices are less affected by recession. "